In [1]:
# header
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

import pandas as pd
import geopandas as gpd
from siuba import *

pd.set_option('display.max_columns', None) 

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/safety_projects/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [2]:
# load crashes
tims = gpd.read_parquet(f'{GCS_FILE_PATH}tims_fsi.parquet')

In [3]:
tims_pedbike = tims[(tims['PEDESTRIAN_ACCIDENT']=='Y') | (tims['BICYCLE_ACCIDENT']=='Y')]

In [4]:
tims_pedbike.info(verbose=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 20779 entries, 0 to 929273
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   CASE_ID              20779 non-null  int32   
 1   ACCIDENT_YEAR        20779 non-null  int32   
 2   COLLISION_DATE       20779 non-null  object  
 3   COLLISION_TIME       20779 non-null  int32   
 4   COLLISION_SEVERITY   20779 non-null  int32   
 5   PCF_VIOL_CATEGORY    20779 non-null  object  
 6   TYPE_OF_COLLISION    20779 non-null  object  
 7   MVIW                 20779 non-null  object  
 8   NUMBER_KILLED        20779 non-null  int32   
 9   NUMBER_INJURED       20779 non-null  int32   
 10  PEDESTRIAN_ACCIDENT  20775 non-null  object  
 11  BICYCLE_ACCIDENT     20779 non-null  object  
 12  LATITUDE             9412 non-null   float64 
 13  LONGITUDE            9412 non-null   float64 
 14  POINT_X              20779 non-null  float64 
 15  POINT_Y   

In [5]:
tims_pedbike.head(10)

,CASE_ID,ACCIDENT_YEAR,COLLISION_DATE,COLLISION_TIME,COLLISION_SEVERITY,PCF_VIOL_CATEGORY,TYPE_OF_COLLISION,MVIW,NUMBER_KILLED,NUMBER_INJURED,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,LATITUDE,LONGITUDE,POINT_X,POINT_Y,geometry
0,8046088,2016,2016-03-11,1825,2,10,G,B,0,1,Y,,NaN,NaN,-122.214510,37.832760,POINT (-194624.642 -18137.757)
1,8051338,2016,2016-03-13,1313,2,10,G,B,0,1,Y,,NaN,NaN,-122.266279,37.812147,POINT (-199228.364 -20321.027)
42,90703926,2018,2018-04-11,704,2,11,G,B,0,1,Y,,37.777168,-122.236931,-122.236847,37.777233,POINT (-196733.011 -24262.688)
71,90101729,2016,2016-01-09,1105,2,10,G,B,0,1,Y,,37.695450,-122.074150,-122.073947,37.695552,POINT (-182606.783 -33666.400)
106,8761600,2018,2018-12-13,1827,2,05,D,G,0,1,,Y,NaN,NaN,-122.130119,37.686680,POINT (-187573.978 -34542.828)
107,8771228,2018,2018-11-13,2009,2,11,B,B,0,1,Y,,NaN,NaN,-122.269623,37.824955,POINT (-199488.151 -18890.610)
108,8778038,2018,2018-12-19,1456,2,11,A,B,0,1,Y,,NaN,NaN,-122.209984,37.770065,POINT (-194389.377 -25114.702)
121,8675427,2018,2018-03-28,2126,2,08,D,G,0,1,,Y,NaN,NaN,-122.275703,37.828224,POINT (-200013.781 -18514.477)
179,90943126,2019,2019-02-27,1205,2,00,C,C,0,6,Y,,37.626389,-122.126060,-122.126068,37.626404,POINT (-187367.545 -41249.803)
226,9152124,2020,2020-07-17,754,2,10,G,B,0,1,Y,,NaN,NaN,-121.930389,37.688572,POINT (-169984.441 -34709.168)


In [6]:
# load bridge buffer areas
bridge_areas = gpd.read_parquet(f'{GCS_FILE_PATH}bridgeareas_clean.parquet')

In [7]:
# spatial join and aggregate crashes to bridge areas
bridges_crashes = gpd.sjoin(bridge_areas, tims_pedbike, how="left")

In [8]:
len(bridges_crashes)

10154

In [9]:
bridges_crashes[bridges_crashes['PEDESTRIAN_ACCIDENT'].notnull()].head(10)

,geometry,BRIDGE_right,DIST_left,CO_left,NAME_left,FAC_left,INTERSEC_left,AADT_left,PM_left,RTE_left,index_right,CASE_ID,ACCIDENT_YEAR,COLLISION_DATE,COLLISION_TIME,COLLISION_SEVERITY,PCF_VIOL_CATEGORY,TYPE_OF_COLLISION,MVIW,NUMBER_KILLED,NUMBER_INJURED,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,LATITUDE,LONGITUDE,POINT_X,POINT_Y
BRIDGE_left,,,,,,,,,,,,,,,,,,,,,,,,,,,
01 0023,"POLYGON ((-343971.563 442005.777, -343971.563 ...",01 0023,1,DN,ROWDY CREEK,U.S. HIGHWAY 101,ROWDY CREEK,6700,39.63,101,67226.0,90962356.0,2019.0,2019-03-31,1042.0,2.0,03,H,J,0.0,1.0,,Y,41.928150,-124.143188,-124.143150,41.928192
01 0064,"POLYGON ((-348366.829 424944.532, -348366.829 ...",01 0064,1,DN,WASHINGTON BLVD OC,WASHINGTON BLVD,U.S. HIGHWAY 101,29500,R27.87,101,66681.0,90568365.0,2017.0,2017-09-20,2008.0,1.0,11,G,B,1.0,0.0,Y,,41.771990,-124.185640,-124.185648,41.771989
01 0064,"POLYGON ((-348366.829 424944.532, -348366.829 ...",01 0064,1,DN,WASHINGTON BLVD OC,WASHINGTON BLVD,U.S. HIGHWAY 101,29500,R27.87,101,67223.0,90951556.0,2019.0,2019-03-15,1540.0,2.0,03,G,B,0.0,1.0,Y,,41.772541,-124.188538,-124.187813,41.772236
02 0163L,"POLYGON ((-199325.957 379709.133, -199329.085 ...",02 0163L,2,SIS,SISKIYOU WAY UC,INTERSTATE 5 NB,SISKIYOU WAY,7150,R18.71,5,850230.0,90467108.0,2017.0,2017-05-22,2145.0,2.0,11,G,B,0.0,1.0,Y,,41.413770,-122.381530,-122.381454,41.413828
04 0054,"POLYGON ((-344091.944 324111.919, -344091.944 ...",04 0054,1,HUM,SEVENTH STREET OC,SEVENTH ST,U.S. HIGHWAY 101,34600,86.1,101,86422.0,9135307.0,2020.0,2020-05-06,1945.0,2.0,08,H,A,0.0,1.0,,Y,40.865700,-124.085274,-124.085632,40.866219
04 0110,"POLYGON ((-355955.821 297363.905, -355955.821 ...",04 0110,1,HUM,SINGLEY ROAD SEPARATION (101/211),US HIGHWAY 101,STATE ROUTE 211,20400,64.29,101,87567.0,90815483.0,2018.0,2018-09-14,2218.0,2.0,09,D,G,0.0,1.0,,Y,40.620708,-124.210121,-124.210121,40.620708
04 0145,"POLYGON ((-352454.888 294730.056, -352454.888 ...",04 0145,1,HUM,FORTUNA OH,US HIGHWAY 101,"S101-MAIN ON&OFF,NCRRA",12800,61.53,101,86337.0,91154853.0,2019.0,2019-08-27,58.0,1.0,11,G,B,1.0,0.0,Y,,40.599411,-124.167847,-124.167839,40.599419
04 0173,"POLYGON ((-346514.790 332557.458, -346514.790 ...",04 0173,1,HUM,HILLER ROAD OC,HILLER ROAD,U.S. HIGHWAY 101,12900,R91.99,101,87778.0,91092609.0,2019.0,2019-10-07,1445.0,2.0,08,G,B,0.0,1.0,Y,,40.941608,-124.114769,-124.114853,40.941685
04 0191,"POLYGON ((-338669.953 327390.200, -338669.953 ...",04 0191,1,HUM,GLENDALE UC (RAMP RD),STATE ROUTE 299,RAMP ROAD,9700,R4.04,299,87980.0,91360289.0,2020.0,2020-11-26,1448.0,2.0,03,G,B,0.0,1.0,Y,,NaN,NaN,-124.021187,40.899677


In [10]:
bridges_crashes >> count (_.PEDESTRIAN_ACCIDENT)

,PEDESTRIAN_ACCIDENT,n
0,,605
1,Y,2373
2,NaN,7176


In [11]:
# need to create dummies for ped and bike - non-"Y" showing up as non-null
import numpy as np

bridges_crashes = bridges_crashes.assign(ped_crash = np.where(bridges_crashes['PEDESTRIAN_ACCIDENT']=="Y",1,0),
                       bike_crash = np.where(bridges_crashes['BICYCLE_ACCIDENT']=="Y",1,0)
                      )

In [12]:
bridges_crashes['pedbike_crash'] = bridges_crashes[["ped_crash", "bike_crash"]].max(axis=1)

In [13]:
bridges_crashes[bridges_crashes['PEDESTRIAN_ACCIDENT'].notnull()].head(10)

,geometry,BRIDGE_right,DIST_left,CO_left,NAME_left,FAC_left,INTERSEC_left,AADT_left,PM_left,RTE_left,index_right,CASE_ID,ACCIDENT_YEAR,COLLISION_DATE,COLLISION_TIME,COLLISION_SEVERITY,PCF_VIOL_CATEGORY,TYPE_OF_COLLISION,MVIW,NUMBER_KILLED,NUMBER_INJURED,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,LATITUDE,LONGITUDE,POINT_X,POINT_Y,ped_crash,bike_crash,pedbike_crash
BRIDGE_left,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01 0023,"POLYGON ((-343971.563 442005.777, -343971.563 ...",01 0023,1,DN,ROWDY CREEK,U.S. HIGHWAY 101,ROWDY CREEK,6700,39.63,101,67226.0,90962356.0,2019.0,2019-03-31,1042.0,2.0,03,H,J,0.0,1.0,,Y,41.928150,-124.143188,-124.143150,41.928192,0,1,1
01 0064,"POLYGON ((-348366.829 424944.532, -348366.829 ...",01 0064,1,DN,WASHINGTON BLVD OC,WASHINGTON BLVD,U.S. HIGHWAY 101,29500,R27.87,101,66681.0,90568365.0,2017.0,2017-09-20,2008.0,1.0,11,G,B,1.0,0.0,Y,,41.771990,-124.185640,-124.185648,41.771989,1,0,1
01 0064,"POLYGON ((-348366.829 424944.532, -348366.829 ...",01 0064,1,DN,WASHINGTON BLVD OC,WASHINGTON BLVD,U.S. HIGHWAY 101,29500,R27.87,101,67223.0,90951556.0,2019.0,2019-03-15,1540.0,2.0,03,G,B,0.0,1.0,Y,,41.772541,-124.188538,-124.187813,41.772236,1,0,1
02 0163L,"POLYGON ((-199325.957 379709.133, -199329.085 ...",02 0163L,2,SIS,SISKIYOU WAY UC,INTERSTATE 5 NB,SISKIYOU WAY,7150,R18.71,5,850230.0,90467108.0,2017.0,2017-05-22,2145.0,2.0,11,G,B,0.0,1.0,Y,,41.413770,-122.381530,-122.381454,41.413828,1,0,1
04 0054,"POLYGON ((-344091.944 324111.919, -344091.944 ...",04 0054,1,HUM,SEVENTH STREET OC,SEVENTH ST,U.S. HIGHWAY 101,34600,86.1,101,86422.0,9135307.0,2020.0,2020-05-06,1945.0,2.0,08,H,A,0.0,1.0,,Y,40.865700,-124.085274,-124.085632,40.866219,0,1,1
04 0110,"POLYGON ((-355955.821 297363.905, -355955.821 ...",04 0110,1,HUM,SINGLEY ROAD SEPARATION (101/211),US HIGHWAY 101,STATE ROUTE 211,20400,64.29,101,87567.0,90815483.0,2018.0,2018-09-14,2218.0,2.0,09,D,G,0.0,1.0,,Y,40.620708,-124.210121,-124.210121,40.620708,0,1,1
04 0145,"POLYGON ((-352454.888 294730.056, -352454.888 ...",04 0145,1,HUM,FORTUNA OH,US HIGHWAY 101,"S101-MAIN ON&OFF,NCRRA",12800,61.53,101,86337.0,91154853.0,2019.0,2019-08-27,58.0,1.0,11,G,B,1.0,0.0,Y,,40.599411,-124.167847,-124.167839,40.599419,1,0,1
04 0173,"POLYGON ((-346514.790 332557.458, -346514.790 ...",04 0173,1,HUM,HILLER ROAD OC,HILLER ROAD,U.S. HIGHWAY 101,12900,R91.99,101,87778.0,91092609.0,2019.0,2019-10-07,1445.0,2.0,08,G,B,0.0,1.0,Y,,40.941608,-124.114769,-124.114853,40.941685,1,0,1
04 0191,"POLYGON ((-338669.953 327390.200, -338669.953 ...",04 0191,1,HUM,GLENDALE UC (RAMP RD),STATE ROUTE 299,RAMP ROAD,9700,R4.04,299,87980.0,91360289.0,2020.0,2020-11-26,1448.0,2.0,03,G,B,0.0,1.0,Y,,NaN,NaN,-124.021187,40.899677,1,0,1


In [15]:
# dissolve, grouping by bridge caracteristics, aggregating counts of encampment work orders 
bridges_crashes_agg = bridges_crashes.dissolve(
    by = ["BRIDGE_left","NAME_left","FAC_left","DIST_left","CO_left"],
    aggfunc = {'NUMBER_KILLED' : 'sum',
               'NUMBER_INJURED' : 'sum',
               'ped_crash' : 'sum',
               'bike_crash' : 'sum',
               'pedbike_crash' : 'sum'
              }
)

In [20]:
bridges_crashes_agg.sample(10)

,BRIDGE_left,NAME_left,FAC_left,geometry,NUMBER_KILLED,NUMBER_INJURED,ped_crash,bike_crash,pedbike_crash
224,04 0089,YAGER CREEK,STATE ROUTE 36,"POLYGON ((-343968.491 287529.900, -343968.491 ...",0.0,0.0,0,0,0
3897,42 0236,OIL CITY (33/5) SEPARATION,STATE ROUTE 33,"POLYGON ((-23970.372 -191403.985, -23970.372 -...",0.0,0.0,0,0,0
6889,54 0659L,IWO JIMA STREET UC,ROUTE 40 E BOUND,"POLYGON ((278061.584 -345261.678, 278063.338 -...",0.0,0.0,0,0,0
7434,55 0416,MAIN STREET UC,STATE ROUTE 55,"POLYGON ((197941.976 -478155.458, 197941.976 -...",1.0,0.0,0,1,1
4673,50 0240R,ROUTE 5/99 SEPARATION,INTERSTATE 5 NB,"POLYGON ((95485.465 -332628.400, 95485.465 -33...",0.0,0.0,0,0,0
7609,55 0771F,S133-S5 CONNECTOR SEPARATION,S133-S5 CONNECTOR,"POLYGON ((208188.272 -480239.226, 208187.474 -...",0.0,0.0,0,0,0
3893,42 0231,JAYNE AVENUE OC,JAYNE AVE,"POLYGON ((-14128.584 -208797.108, -14128.584 -...",0.0,0.0,0,0,0
3300,37 0252K,PAGE MILL ROAD UC,NB I280 ON-RAMP,"POLYGON ((-191436.504 -67464.284, -191436.500 ...",0.0,1.0,0,1,1
1113,15 0068,HUSTED ROAD OC,HUSTED ROAD,"POLYGON ((-184000.549 125679.442, -184000.549 ...",0.0,0.0,0,0,0
6094,53 1763,ARLETA & SHELDON UC,SR 170,"POLYGON ((146597.670 -419783.090, 146597.670 -...",0.0,0.0,0,0,0


In [24]:
bridges_crashes_agg.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   index           8972 non-null   int64   
 1   BRIDGE_left     8972 non-null   object  
 2   NAME_left       8972 non-null   object  
 3   FAC_left        8972 non-null   object  
 4   geometry        8972 non-null   geometry
 5   NUMBER_KILLED   8972 non-null   float64 
 6   NUMBER_INJURED  8972 non-null   float64 
 7   ped_crash       8972 non-null   int64   
 8   bike_crash      8972 non-null   int64   
 9   pedbike_crash   8972 non-null   int64   
dtypes: float64(2), geometry(1), int64(4), object(3)
memory usage: 701.1+ KB


In [21]:
# fix index
bridges_crashes_agg = bridges_crashes_agg.reset_index()

In [18]:
# save out as geoparquet 
shared_utils.utils.geoparquet_gcs_export(bridges_crashes_agg, GCS_FILE_PATH, "pedcrashes_agg")